In [1]:
using StringEncodings
using DataStructures

In [2]:
tokenType = Union{Int, Char}
byteVec = Base.CodeUnits{UInt8, String}
mergeDict = Union{OrderedDict{Tuple{byteVec, byteVec}, Tuple{byteVec, Int}}, OrderedDict{Tuple{Char, Char}, Int}}

Union{OrderedDict{Tuple{Base.CodeUnits{UInt8, String}, Base.CodeUnits{UInt8, String}}, Tuple{Base.CodeUnits{UInt8, String}, Int64}}, OrderedDict{Tuple{Char, Char}, Int64}}

In [3]:
mutable struct BPETokenizer
    mergeDict::mergeDict
    vocabDict::OrderedDict{Int, byteVec}
    type_::tokenType
end

In [4]:
mutable struct TrainingParams
    vocab_size::Int
    n_merges::Int
    null_token::String
    null_byte::byteVec

    function TrainingParams(vocab_size, n_merges, null_token)
        null_byte = transcode(UInt8, string(null_token))
        return new(vocab_size, n_merges, null_token, null_byte)
    end
end

In [5]:
# multiple dispatches of the encoding_without_merging function
function encode_without_merging(input::String)::Vector{byteVec}
    vec_byteVec = Vector{byteVec}()
    text_tokens = input |> collect
    for token in text_tokens
        push!(vec_byteVec, transcode(UInt8, string(token))) 
    end
    return vec_byteVec 
end

encode_without_merging (generic function with 1 method)

In [6]:
# multiple dispatches of the get_stats function
function get_stats(ids::Vector{byteVec})::OrderedDict{Tuple{byteVec, byteVec}, Int}
    stats = OrderedDict{Tuple{byteVec,byteVec}, Int}()
    for i in 1:(length(ids) - 1)
        pair = (ids[i], ids[i+1])
        stats[pair] = get(stats, pair, 0) + 1
    end
    return stats
end

function get_stats(ids::Vector{Vector{UInt8}})::OrderedDict{Tuple{Vector{UInt8}, Vector{UInt8}}, Int}
    stats = OrderedDict{Tuple{UInt8, UInt8}, Int}()
    for i in 1:(length(ids) - 1)
        pair = (ids[i], ids[i+1])
        stats[pair] = get(stats, pair, 0) + 1
    end
    return stats
end


get_stats (generic function with 2 methods)

In [7]:
# multiple dispatches of the get_top_pair function
function get_top_pair(stats::OrderedDict{Tuple{byteVec, byteVec}, Int})::Tuple{byteVec, byteVec}
    top_val = maximum(values(stats))
    if top_val > 1
        for (key, val) in stats
            if val == top_val
                return key
            end
        end
    else 
        null_code = transcode(UInt8, string("<NULL>"))
        return (null_code, null_code)
    end
end

function get_top_pair(stats::Dict{Tuple{UInt8, UInt8}, Int})::Tuple{UInt8, UInt8}
    top_val = maximum(values(stats))
    if top_val > 1
        for (key, val) in stats
            if val == top_val
                return key
            end
        end
    else 
        return (-1, -1)
    end
end

get_top_pair (generic function with 2 methods)

In [8]:
# multiple dispatches of the merge_pair function
function merge_pair(tokens::Vector{byteVec}, pair::Tuple{byteVec, byteVec}, new_token::byteVec)::Vector{byteVec}
    newTokens = byteVec[]
    i = 1
    while i < length(tokens) - 1
        if i < length(tokens) && pair[1] == tokens[i] && pair[2] == tokens[i+1]
            push!(newTokens, new_token)
            i += 2
        else
            push!(newTokens, tokens[i])
            i += 1
        end
    end
    return newTokens
end 

function merge_pair(ids::Vector{Int}, pair::Tuple{Int, Int}, idx::Int)::Vector{Int}
    newIds = Int[]
    i = 1
    while i < length(ids) - 1
        if i < length(ids) && pair[1] == ids[i] && pair[2] == ids[i+1]
            push!(newIds, idx)
            i += 2
        else
            push!(newIds, ids[i])
            i += 1
        end
    end
    return newIds
end 

    

merge_pair (generic function with 2 methods)

In [9]:
transcode(UInt8, string("<NULL>"))

6-element Base.CodeUnits{UInt8, String}:
 0x3c
 0x4e
 0x55
 0x4c
 0x4c
 0x3e

In [10]:
Char(255)

'ÿ': Unicode U+00FF (category Ll: Letter, lowercase)

In [11]:
# multiple dispatches for the build_vocab function
function build_vocab(merge_dict::OrderedDict{Tuple{byteVec, byteVec}, Tuple{byteVec, Int}})::OrderedDict{Int, byteVec}
    vocab = OrderedDict{Int, byteVec}()
    for idx in 0:255
        vocab[idx] = transcode(UInt8, string(Char(idx))) 
    end
    for ((p0, p1), (merged_token, idx)) in merge_dict
        vocab[idx] = merged_token
    end
    return vocab
end

function build_vocab(merge_dict::OrderedDict{Tuple{Int, Int}, Int})::OrderedDict{Int, String}
    vocab = OrderedDict{Int, String}()
    for idx in 0:255
        vocab[idx] = Char(idx)
    end
    for ((p0, p1), idx) in merge_dict
        vocab[idx] = vocab[p0] * vocab[p1]
    end
    return vocab
end

build_vocab (generic function with 2 methods)

In [30]:
# multiple dispatches for the train function
function train(tokenizer::BPETokenizer, params::TrainingParams, tokens::Vector{byteVec})
    merges = OrderedDict{Tuple{byteVec, byteVec}, Tuple{byteVec, Int}}()
    n_merges = params.n_merges
    for i in 1:n_merges
        stats = get_stats(tokens)
        top_pair = get_top_pair(stats)
        if top_pair[1] == params.null_byte && top_pair[2] == params.null_byte
            break
        end
        idx = 256 + i
        new_token = transcode(UInt8, String(vcat(top_pair[1], top_pair[2])))
        tokens = merge_pair(tokens, top_pair, new_token)
        merges[top_pair] = (new_token, idx)
        @info "Merging $top_pair into a new token $new_token"  
    end
    vocab = build_vocab(merges)
    tokenizer.mergeDict = merges
    tokenizer.vocabDict = vocab
end

train (generic function with 1 method)

In [13]:
# multiple dispatches of the encode_tokens function
function encode_tokens(tokenizer::BPETokenizer, input::String, type_::Int)::Vector{byteVec}
    tokens = encode_without_merging(input, type_)
    merges = tokenizer.mergeDict
    while length(tokens) >= 2
        stats = get_stats(tokens)
        pair = findmin([get(merges, p, Inf) for p in keys(stats)])
        if pair[2] ∉ keys(merges)
            break
        end
        idx = merges[pair]
        tokens = merge_pair(tokens, pair, idx)
    end
    return tokens
end  

function encode_tokens(tokenizer::BPETokenizer, input::String, type_::Char)::Vector{Int}
    tokens = encode_without_merging(input, type_)
    merges = tokenizer.mergeDict
    while length(tokens) >= 2
        stats = get_stats(tokens)
        pair = findmin([get(merges, p, Inf) for p in keys(stats)])
        if pair[2] ∉ keys(merges)
            break
        end
        idx = merges[pair]
        tokens = merge_pair(tokens, pair, idx)
    end
    return tokens
end  

encode_tokens (generic function with 2 methods)

In [14]:
# multiple dispatches of decode_tokens function
function decode_tokens(tokenizer::BPETokenizer, ids::Vector{Int}, type_::Char)::String
    @assert typeof(type_) == Char "Not type of Char"
    vocab = tokenizer.vocabDict
    tokens = join([vocab[idx] for idx in ids])
    return tokens
end

function decode_tokens(tokenizer::BPETokenizer, ids::Vector{Int}, type_::Int)::String
    @assert typeof(type_) == Int64 "Not type of Int64"
    vocab = tokenizer.vocabDict
    tokens = join([vocab[idx] for idx in ids])
    @show tokens
    code_units = codeunits(tokens)
    code_points = [Char(code_unit) for code_unit in code_units]
    text = String(code_points)
    return text
end

decode_tokens (generic function with 2 methods)

In [15]:
input = """普通の言葉と簡単な文章を使って書くようにしています。

この種の文章は読みやすく、読みやすいものであればあるほど、読者はより深くその内容に引き込まれます。彼らがあなたの散文に費やすエネルギーが減れば減るほど、あなたのアイデアのために多くの時間が残されるでしょう。
வணக்கம், நீங்கள் எப்படி இருக்கிறீர்கள் ? I try to write using ordinary words and simple sentences.

That kind of writing is easier to read, and the easier something is to read, the more deeply readers will engage with it. The less energy they expend on your prose, the more they'll have left for your ideas.

And the further they'll read. Most readers' energy tends to flag part way through an article or essay. If the friction of reading is low enough, more keep going till the end.

There's an Italian dish called saltimbocca, which means "leap into the mouth." My goal when writing might be called saltintesta: the ideas leap into your head and you barely notice the words that got them there.

It's too much to hope that writing could ever be pure ideas. You might not even want it to be. But for most writers, most of the time, that's the goal to aim for. The gap between most writing and pure ideas is not filled with poetry.

Plus it's more considerate to write simply. When you write in a fancy way to impress people, you're making them do extra work just so you can seem cool. It's like trailing a long train behind you that readers have to carry.

And remember, if you're writing in English, that a lot of your readers won't be native English speakers. Their understanding of ideas may be way ahead of their understanding of English. So you can't assume that writing about a difficult topic means you can use difficult words.

Of course, fancy writing doesn't just conceal ideas. It can also conceal the lack of them. That's why some people write that way, to conceal the fact that they have nothing to say. Whereas writing simply keeps you honest. If you say nothing simply, it will be obvious to everyone, including you.

Simple writing also lasts better. People reading your stuff in the future will be in much the same position as people from other countries reading it today. The culture and the language will have changed. It's not vain to care about that, any more than it's vain for a woodworker to build a chair to last.

Indeed, lasting is not merely an accidental quality of chairs, or writing. It's a sign you did a good job.

But although these are all real advantages of writing simply, none of them are why I do it. The main reason I write simply is that it offends me not to. When I write a sentence that seems too complicated, or that uses unnecessarily intellectual words, it doesn't seem fancy to me. It seems clumsy.

There are of course times when you want to use a complicated sentence or fancy word for effect. But you should never do it by accident.

The other reason my writing ends up being simple is the way I do it. I write the first draft fast, then spend days editing it, trying to get everything just right. Much of this editing is cutting, and that makes simple writing even simpler."""

"普通の言葉と簡単な文章を使って書くようにしています。\n\nこの種の文章は読みやすく、読みやすいものであればあるほど、読者はより深くその内容に引き込まれます。彼らがあなたの散文に費やすエネルギーが減れば減るほど、あなたのアイデアのために多くの時間が残されるでしょう。\nவணக்கம், நீங்கள் எப்படி இருக்கிறீர்கள் ? I try to write using ordinary words and simple sentences.\n\nThat kind of writing is easier to read" ⋯ 2417 bytes ⋯ "uld never do it by accident.\n\nThe other reason my writing ends up being simple is the way I do it. I write the first draft fast, then spend days editing it, trying to get everything just right. Much of this editing is cutting, and that makes simple writing even simpler."

In [16]:
enc_vec = encode_without_merging(input)

2957-element Vector{Base.CodeUnits{UInt8, String}}:
 [0xe6, 0x99, 0xae]
 [0xe9, 0x80, 0x9a]
 [0xe3, 0x81, 0xae]
 [0xe8, 0xa8, 0x80]
 [0xe8, 0x91, 0x89]
 [0xe3, 0x81, 0xa8]
 [0xe7, 0xb0, 0xa1]
 [0xe5, 0x8d, 0x98]
 [0xe3, 0x81, 0xaa]
 [0xe6, 0x96, 0x87]
 [0xe7, 0xab, 0xa0]
 [0xe3, 0x82, 0x92]
 [0xe4, 0xbd, 0xbf]
 ⋮
 [0x76]
 [0x65]
 [0x6e]
 [0x20]
 [0x73]
 [0x69]
 [0x6d]
 [0x70]
 [0x6c]
 [0x65]
 [0x72]
 [0x2e]

In [17]:
tokenizer = BPETokenizer(OrderedDict{Tuple{byteVec, byteVec}, Tuple{byteVec, Int}}(), OrderedDict{Int, byteVec}(), Char(' '))

BPETokenizer(OrderedDict{Tuple{Base.CodeUnits{UInt8, String}, Base.CodeUnits{UInt8, String}}, Tuple{Base.CodeUnits{UInt8, String}, Int64}}(), OrderedDict{Int64, Base.CodeUnits{UInt8, String}}(), ' ')

In [18]:
vocab_size = 300
params = TrainingParams(vocab_size, (vocab_size - 256), "<NULL>")

TrainingParams(300, 44, "<NULL>", UInt8[0x3c, 0x4e, 0x55, 0x4c, 0x4c, 0x3e])

In [31]:
train(tokenizer, params, enc_vec) 

[ Info: Merging (UInt8[0x65], UInt8[0x20]) into a new token UInt8[0x65, 0x20]
[ Info: Merging (UInt8[0x20], UInt8[0x74]) into a new token UInt8[0x20, 0x74]
[ Info: Merging (UInt8[0x69], UInt8[0x6e]) into a new token UInt8[0x69, 0x6e]
[ Info: Merging (UInt8[0x74], UInt8[0x20]) into a new token UInt8[0x74, 0x20]
[ Info: Merging (UInt8[0x73], UInt8[0x20]) into a new token UInt8[0x73, 0x20]
[ Info: Merging (UInt8[0x20, 0x74], UInt8[0x68]) into a new token UInt8[0x20, 0x74, 0x68]
[ Info: Merging (UInt8[0x69], UInt8[0x74]) into a new token UInt8[0x69, 0x74]
[ Info: Merging (UInt8[0x69, 0x6e], UInt8[0x67]) into a new token UInt8[0x69, 0x6e, 0x67]
[ Info: Merging (UInt8[0x65], UInt8[0x61]) into a new token UInt8[0x65, 0x61]
[ Info: Merging (UInt8[0x6f], UInt8[0x75]) into a new token UInt8[0x6f, 0x75]
[ Info: Merging (UInt8[0x6f], UInt8[0x20]) into a new token UInt8[0x6f, 0x20]
[ Info: Merging (UInt8[0x65], UInt8[0x72]) into a new token UInt8[0x65, 0x72]
[ Info: Merging (UInt8[0x61], UInt8[0x6e

OrderedDict{Int64, Base.CodeUnits{UInt8, String}} with 300 entries:
  0  => [0x00]
  1  => [0x01]
  2  => [0x02]
  3  => [0x03]
  4  => [0x04]
  5  => [0x05]
  6  => [0x06]
  7  => [0x07]
  8  => [0x08]
  9  => [0x09]
  10 => [0x0a]
  11 => [0x0b]
  12 => [0x0c]
  13 => [0x0d]
  14 => [0x0e]
  15 => [0x0f]
  16 => [0x10]
  17 => [0x11]
  18 => [0x12]
  19 => [0x13]
  20 => [0x14]
  21 => [0x15]
  22 => [0x16]
  23 => [0x17]
  24 => [0x18]
  ⋮  => ⋮

In [33]:
tokenizer.vocabDict[260]

2-element Base.CodeUnits{UInt8, String}:
 0x74
 0x20